In [1]:
# Data processing
import pandas as pd

test_prompt_data = pd.read_csv("/data/yingfei/cancer_data/llm_prompt_data/test_prompt_data_task1_simple.csv")
print(test_prompt_data.shape)
test_prompt_data.head()

(2312, 3)


,cell_id,prompt,answer
0,ACH-000006,Think step by step and decide in a single word...,Sensitive
1,ACH-000006,Think step by step and decide in a single word...,Resistant
2,ACH-000006,Think step by step and decide in a single word...,Resistant
3,ACH-000006,Think step by step and decide in a single word...,Sensitive
4,ACH-000006,Think step by step and decide in a single word...,Sensitive


In [2]:
test_prompt_data.iloc[219:,:] # N= 219

,cell_id,prompt,answer
219,ACH-000139,Think step by step and decide in a single word...,Sensitive
220,ACH-000139,Think step by step and decide in a single word...,Sensitive
221,ACH-000139,Think step by step and decide in a single word...,Resistant
222,ACH-000139,Think step by step and decide in a single word...,Resistant
223,ACH-000139,Think step by step and decide in a single word...,Resistant
...,...,...,...
2307,ACH-001654,Think step by step and decide in a single word...,Sensitive
2308,ACH-001654,Think step by step and decide in a single word...,Sensitive
2309,ACH-001654,Think step by step and decide in a single word...,Sensitive
2310,ACH-001654,Think step by step and decide in a single word...,Resistant


In [3]:
test_samples = test_prompt_data.iloc[:200,:].copy()
test_samples['output70'] = [None,]*200
# test_samples = pd.read_csv('./data/sensitivity/test_fewshot_simple.csv')

In [4]:
test_samples.head()

,cell_id,prompt,answer,output70
0,ACH-000006,Think step by step and decide in a single word...,Sensitive,None
1,ACH-000006,Think step by step and decide in a single word...,Resistant,None
2,ACH-000006,Think step by step and decide in a single word...,Resistant,None
3,ACH-000006,Think step by step and decide in a single word...,Sensitive,None
4,ACH-000006,Think step by step and decide in a single word...,Sensitive,None


In [5]:
fewshot_set = test_prompt_data.iloc[219:,:] #1. not appear in train data; 2. not to inference (test data top 200 -> inf)
fewshot_set.shape

(2093, 3)

In [14]:
torch.cuda.device_count()

4

In [1]:
import torch
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer
model_dir = "/data/yingfei/models/llm/llama2/llama/llama-2-7b-chat-hf" #"/data/yingfei/models/llm/meditron-7b"

# model = LlamaForCausalLM.from_pretrained(model_dir, device_map = "auto", low_cpu_mem_usage=True)#.to(DEVICE)
# tokenizer = LlamaTokenizer.from_pretrained(model_dir)#.to(model.device)

# torch.cuda.set_per_process_memory_fraction(1.0)
# model = LlamaForCausalLM.from_pretrained(model_dir, device_map = {"":0})
tokenizer = LlamaTokenizer.from_pretrained(model_dir)

device_map = 'auto'

model = LlamaForCausalLM.from_pretrained(model_dir, device_map=device_map)

# from inferer import Inferer
# medalpaca = Inferer(
#     model_name="medalpaca/medalapca-7b", 
#     prompt_template="path/to/your/prompt_template.json",
#     base_model=None,  # Optional, specify if using LoRA or another base model
#     model_max_length=512,  # Optional, can be adjusted based on your input needs
#     load_in_8bit=False,  # Optional, set to True if using a quantized model
#     torch_dtype=torch.float16,  # Optional, can be adjusted based on your precision needs
#     peft=False  # Optional, set to True if the model was trained with PEFT or in 8bit
# )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
import numpy as np

def get_few_shot(fewshot_set, num=5):
    fewshot = ''
    for i in range(num):#len(fewshot_set)):
        idx = np.random.randint(0,len(fewshot_set))
        answer = fewshot_set.iloc[idx].answer
        question = fewshot_set.iloc[idx].prompt.split("[Reasoning].")[1].replace('?',answer)#.replace("\n", "")
        fewshot += f'\n\nExample {i}:\n'
        fewshot += question#.strip()
    
    return fewshot.strip()

In [12]:
for idx in range(200):
    row = test_samples.iloc[idx,:]
    if not pd.isna(row.output70): continue
    print(f'PROCESSING CELLINE: {row.cell_id}   ', idx)
    prompt = row.prompt

    pipeline = transformers.pipeline("text-generation", model=model,
                                    tokenizer=tokenizer, temperature = 0.0,
                                    torch_dtype=torch.float16, device_map="auto",
                                    return_full_text=False)
    instruction = "Think step by step and decide in a single word reflecting the drug sensitivity of the drug on the cell line: [Sensitive/Resistant]"
    prompt = prompt.split("[Reasoning].")[1]
    #fewshot = get_few_shot(fewshot_set, num=5)
    #prompt_content = f"<s>[INST] <<SYS>>{instruction}<</SYS>> <<EXAMPLE>>{fewshot}<</EXAMPLE>> {prompt}[/INST]"
    prompt_content = f"<s>[INST] <<SYS>>{instruction}<</SYS>> {prompt}[/INST]"
    
    print(prompt_content)
    # Run prompt and pipeline
    sequences = pipeline(prompt_content, truncation=True, max_length=20480)
    # print(result[0]['generated_text']) # Not in one word??? !!!
    output = ""
    for seq in sequences:
        new_output = seq['generated_text']
        print(new_output)
        output += new_output
    

    test_samples.loc[idx,'output70'] = output.strip()
    test_samples.to_csv('meditron_7b_sens_simp_zs.csv', index=None)
    print()
    print(f'GROUND TRUTH: {row.answer}')
    print('*'*20)

PROCESSING CELLINE: ACH-000006    0
<s>[INST] <<SYS>>Think step by step and decide in a single word reflecting the drug sensitivity of the drug on the cell line: [Sensitive/Resistant]<</SYS>> 
Drug and cell line mutations: 
The drug is JW-7-24-1. The drug SMILES structure is COC1=CC(=CC(=C1)C2=CC3=C4C(=CN=C3C=C2)C=CC(=O)N4C5=CC(=C(C=C5)N6CCNCC6)C(F)(F)F)OC. Drug target is LCK. Drug target pathway is Other, kinases.
The mutations of the cell line are ASXL1, FLT3, MAP3K1, NOTCH1, TP53, U2AF1.
Drug Sensitivity: ?[/INST]


RuntimeError: CUDA out of memory. Tried to allocate 502.00 MiB (GPU 0; 31.75 GiB total capacity; 10.32 GiB already allocated; 447.50 MiB free; 31.75 GiB allowed; 10.50 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import pandas as pd

df_result = pd.read_csv('test_output.csv')
df_result.head()

In [ ]:
pred_lst = []
for x in df_result.output70:
    pre = x.find('Resistant')
    if pre != -1:
        pred_lst.append('Resistant')
    else:
        pred_lst.append('Sensitive')

In [ ]:
pred_lst[:5]

In [ ]:
corr = [1 for idx, x in enumerate(df_result.answer) if x == pred_lst[idx]]

In [ ]:
sum(corr)/len(df_result)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
cm = confusion_matrix(df_result.answer.tolist(), pred_lst, labels=["Resistant", "Sensitive"])

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Resistant", "Sensitive"])

In [ ]:
disp.plot()